<a href="https://colab.research.google.com/github/Scalable-DNN/pretrained-models/blob/main/cifar10_mobilenetV2alpha1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Feb  5 14:43:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install larq

     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=14074e7061ab4a71cdeaaecd860f363959833afca588dc06704a98e41d8761b7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [3]:
import os, sys
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications as kapp, models
from tensorflow.keras.datasets import cifar10
import larq as lq

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1)

In [6]:
x_test  = tf.image.resize(x_test, (96, 96), 'bicubic').numpy()
x_train = tf.image.resize(x_train, (96, 96), 'bicubic').numpy()
x_val = tf.image.resize(x_val, (96, 96), 'bicubic').numpy()

In [7]:
print("X_train.shape: {}\tY_train.shape: {}".format(x_train.shape, y_train.shape))
print("X_val.shape:   {}\tY_val.shape:   {}".format(x_val.shape, y_val.shape))
print("X_test.shape:  {}\tY_test.shape:  {}".format(x_test.shape, y_test.shape))

X_train.shape: (45000, 96, 96, 3)	Y_train.shape: (45000, 1)
X_val.shape:   (5000, 96, 96, 3)	Y_val.shape:   (5000, 1)
X_test.shape:  (10000, 96, 96, 3)	Y_test.shape:  (10000, 1)


In [8]:
to_categorical = keras.utils.to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [93]:
DataGen = keras.preprocessing.image.ImageDataGenerator

transformations = {
    'rotation_range': 30,
    # 'featurewise_std_normalization': True,
    # 'featurewise_center': True,
    'zoom_range': .5,
    'horizontal_flip': True,
    # 'zca_whitening': True, 
    'width_shift_range': 10 / 32, 
    'height_shift_range': 10 /32, 
    # 'brightness_range': (0, .2), 
    'rescale': 1/255.,
    'shear_range': .5,
    # 'preprocessing_function': kapp.mobilenet_v2.preprocess_input
}

train_generator = DataGen(**transformations)
val_generator   = DataGen(rescale=1/255., rotation_range=50, horizontal_flip=True, zoom_range=.5,
)
test_generator  = DataGen(rescale=1/255.)

In [10]:
# train_generator.fit(x_train)
# val_generator.fit(x_val)
# test_generator.fit(x_test)

In [11]:
# core_model = kapp.MobileNetV2()
# core_model.summary()

In [39]:
input_shape = (96, 96, 3)
fine_tune = True
# core_model = kapp.EfficientNetB0(x_train.shape[1:], weights='imagenet', include_top=False, classes=y_train.shape[-1])
core_model = kapp.MobileNetV2(include_top=False, input_shape=input_shape, classes=y_train.shape[-1], weights='imagenet')
core_model.trainable = fine_tune

inputs = keras.layers.Input(shape=input_shape)

# norm_layer = keras.layers.experimental.preprocessing.Normalization()
# mean = np.array([127.5] * 3)
# # print(mean)
# var = mean ** 2
# # Scale inputs to [-1, +1]
# x = norm_layer(inputs)
# norm_layer.set_weights([mean, var])

# # x = core_model.output
# x = core_model(x, training=False)
x = core_model(inputs, training=fine_tune)
x = keras.layers.Dropout(.3)(x)
x = keras.layers.GlobalAveragePooling2D(name='global_average_pooling')(x)
# x = keras.layers.Dense(512, activation='relu')(x)
# x = keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(x)
# x = keras.layers.Dropout(.35)(x)
output = keras.layers.Dense(y_train.shape[-1], name='predictions', activation='softmax')(x)
# x = keras.layers.Reshape((1, 1, -1))(x)
# x = keras.layers.Dropout(.2, name='dropout', )(x)
# x = keras.layers.Conv2D(filters = y_train.shape[-1], kernel_size=1, name='logits')(x)
# x = keras.layers.Flatten()(x)
# output = keras.layers.Activation('softmax', name='predictions')(x)

# model = models.Model(inputs=core_model.input, outputs=output)
model = models.Model(inputs=inputs, outputs=output)

model.summary()
print(len(model.layers))
# lq.models.summary(model)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 3, 1280)        0         
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                12810     
Total params: 2,270,794
Trainable params: 2,236,682
Non-trainable params: 34,112
_________________________________________________________________
5


In [74]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(core_model.layers))
core_model.trainable = True

# Fine-tune from this layer onwards
# fine_tune_at = 107
# fine_tune_at = 81
fine_tune_at = 0
print(type(core_model.layers[fine_tune_at]))

# Freeze all the layers before the `fine_tune_at` layer
for layer in core_model.layers[:fine_tune_at + 1]:
  layer.trainable =  False

model.summary()

Number of layers in the base model:  154
<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 3, 1280)        0         
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                12810     
Total params: 2,270,794
Trainable params: 2,236,682
Non-trainable params: 34,112
_________________________________________________________________


In [96]:
batch_size = 256
epochs = 1000
initial_learning_rate = 0.0008
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

optimzer = keras.optimizers.Adam(initial_learning_rate)
callbacks = [
              keras.callbacks.EarlyStopping('val_accuracy', patience=10, verbose=1, restore_best_weights=True),
              keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=.5, patience=6, verbose=1),
              keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4', monitor='val_accuracy', save_best_only=True)
]

## Transfer Learning

In [68]:
# model_2 = keras.models.load_model('/content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_2')
model_2 = keras.models.load_model('mobilenetv2_temp')

In [77]:
model.compile(optimizer=optimzer, loss='categorical_crossentropy', metrics=['accuracy'], )

In [97]:
model.fit(
    train_generator.flow(x_train, y_train, batch_size),
    epochs=epochs,
    shuffle=True,
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=val_generator.flow(x_val, y_val, batch_size),
    validation_steps=x_val.shape[0] // batch_size,
    callbacks=callbacks,
)

Epoch 1/1000
175/175 [==============================] - 170s 971ms/step - loss: 0.2922 - accuracy: 0.8989 - val_loss: 0.4300 - val_accuracy: 0.8581


INFO:tensorflow:Assets written to: /content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4/assets


Epoch 2/1000
175/175 [==============================] - 169s 962ms/step - loss: 0.2844 - accuracy: 0.8994 - val_loss: 0.4244 - val_accuracy: 0.8577
Epoch 3/1000
175/175 [==============================] - 168s 955ms/step - loss: 0.2734 - accuracy: 0.9032 - val_loss: 0.4100 - val_accuracy: 0.8614


INFO:tensorflow:Assets written to: /content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4/assets


...

Epoch 7/1000
175/175 [==============================] - 166s 947ms/step - loss: 0.2555 - accuracy: 0.9104 - val_loss: 0.4166 - val_accuracy: 0.8590
Epoch 8/1000
175/175 [==============================] - 168s 960ms/step - loss: 0.2576 - accuracy: 0.9105 - val_loss: 0.4203 - val_accuracy: 0.8563
Epoch 9/1000
175/175 [==============================] - 168s 955ms/step - loss: 0.2528 - accuracy: 0.9107 - val_loss: 0.4074 - val_accuracy: 0.8614
Epoch 10/1000
175/175 [==============================] - 170s 969ms/step - loss: 0.2451 - accuracy: 0.9140 - val_loss: 0.4320 - val_accuracy: 0.8586
Epoch 11/1000
175/175 [==============================] - 168s 956ms/step - loss: 0.2486 - accuracy: 0.9140 - val_loss: 0.4176 - val_accuracy: 0.8643
Epoch 12/1000
175/175 [==============================] - 170s 970ms/step - loss: 0.2339 - accuracy: 0.9182 - val_loss: 0.4222 - val_accuracy: 0.8641

Epoch 00012: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 13/1000
175/175 [=====

INFO:tensorflow:Assets written to: /content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_4/assets


Epoch 14/1000
175/175 [==============================] - 169s 961ms/step - loss: 0.2355 - accuracy: 0.9179 - val_loss: 0.4252 - val_accuracy: 0.8616
Epoch 15/1000
175/175 [==============================] - 166s 945ms/step - loss: 0.2381 - accuracy: 0.9157 - val_loss: 0.4384 - val_accuracy: 0.8544
Epoch 16/1000
175/175 [==============================] - 164s 936ms/step - loss: 0.2388 - accuracy: 0.9160 - val_loss: 0.4374 - val_accuracy: 0.8536
Epoch 17/1000
175/175 [==============================] - 165s 938ms/step - loss: 0.2321 - accuracy: 0.9177 - val_loss: 0.4142 - val_accuracy: 0.8608
Epoch 18/1000
175/175 [==============================] - 165s 942ms/step - loss: 0.2332 - accuracy: 0.9178 - val_loss: 0.4136 - val_accuracy: 0.8625
Epoch 19/1000
175/175 [==============================] - 166s 944ms/step - loss: 0.2315 - accuracy: 0.9195 - val_loss: 0.4025 - val_accuracy: 0.8670

Epoch 00019: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 20/1000
175/175 [==

KeyboardInterrupt: ignored

In [98]:
model.evaluate(test_generator.flow(x_test, y_test, batch_size=batch_size))

40/40 [==============================] - 6s 155ms/step - loss: 0.2390 - accuracy: 0.9166


[0.2390439659357071, 0.9165999889373779]

In [99]:
model.save('mobilnetV2alpha1')

INFO:tensorflow:Assets written to: mobilnetV2alpha1/assets


INFO:tensorflow:Assets written to: mobilnetV2alpha1/assets
